# Analise Exploratória de Dados - Despesa Pública

Este notebook tem como objetivo realizar a conexão com o banco de dados e executar as queries de investigação definidas para o desafio.

# Sumário

- [1. Configuração e Conexão](#1-configuração-e-conexão)
- [2. Análise Exploratória](#2-análise-exploratória)
- [3. Detecção de Anomalias](#3-detecção-de-anomalias)
- [4. Visualização Detalhada dos Resultados](#4-visualização-detalhada-dos-resultados)


## 1. Configuração e Conexão

In [1]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

# Adicionar diretório raiz ao path para importar src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.database import get_db_engine, run_query, load_query

# Configurar estilo dos gráficos
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
# Testar conexão
engine = get_db_engine()
if engine:
    print("Conexão estabelecida com sucesso!")
else:
    print("Falha na conexão.")

Conexão estabelecida com sucesso!


## 2. Visão Geral dos Dados
Para ter uma noção da dimensão dos dados que serão trabalhos, foi feita a consulta abaixo sobre as principais tabelas do fluxo de despeas públicas:

In [3]:
query_volumetria = load_query('queries_base/volumetria_geral.sql')

df_volumetria = run_query(query_volumetria)
df_volumetria

,tabela,total
0,contrato,500
1,empenho,497
2,liquidacao_nota_fiscal,457
3,pagamento,497


A partir desse resultado, já é possível identificar diferenças nos valores entre tabelas que devem ter a mesma dimensão, como a tabela de pagamentos e a tabela de liquidações.

## 3. Investigação de Anomalias (Questão 2)
Através das consultas ao banco de dados, será possível identificar ocasionais inconsistências que podem indicar problemas na gestão dos recursos públicos. Algums problemas já supostos através da busca direto no banco são:
- Quebra do Fluxo da Despesa Pública
- Pagamento com valor maior que o de contrato e empenho
- Datas inconsistentes para as etapas do fluxo

Abaixo executamos as consultas que foram identificadas para detectar possíveis inconsistências.

### 3.1 Validação do fluxo da despesa pública

Busca-se validar o fluxo da despesa pública, verificando se os pagamentos foram realizados corretamente. Se há empenhos sem contratos, pagamentos sem empenhos, ou pagamentos que não foram liquidados.

Empenhos que não possuem nenhum contrato associado indica alguma irregularidade na execução orçamentária.

In [4]:
query_empenho_sem_contrato = load_query('queries_base/empenhos_sem_contratos.sql')

df_empenho_sem_contrato = run_query(query_empenho_sem_contrato)
print(f"Empenhos sem contratos: {len(df_empenho_sem_contrato)}")
df_empenho_sem_contrato.head()

Empenhos sem contratos: 0


,id_empenho,valor,data_empenho


Contratos que não possuem nenhum empenho associado podem indicar cadastros orfãos ou falta de execução.

In [5]:
query_contratos_sem_empenho = load_query('queries_base/contratos_sem_empenho.sql')

df_contratos_sem_empenho = run_query(query_contratos_sem_empenho)
print(f"Contratos sem empenho: {len(df_contratos_sem_empenho)}")
df_contratos_sem_empenho.head()

Contratos sem empenho: 3


,objeto,data,valor
0,Fornecimento de Papel A4,2024-01-03,15188.58
1,Merenda Escolar - Lote 1,2024-01-03,39185.84
2,Consultoria Financeira,2024-01-10,7046.77


Liquidações sem empenho ferem a lógica do processo de despesa pública, pois a liquidação é a etapa que verifica se o serviço foi prestado ou o produto foi entregue, e o empenho é a etapa que reserva o valor para o pagamento.

In [6]:
query_liquidacoes_sem_empenho = load_query('queries_base/liquidacoes_sem_empenho.sql')

df_liquidacoes_sem_empenho = run_query(query_liquidacoes_sem_empenho)
print(f"Liquidações sem empenho: {len(df_liquidacoes_sem_empenho)}")
df_liquidacoes_sem_empenho.head()


Liquidações sem empenho: 0


,id_liquidacao_empenhonotafiscal,valor_liquidacao,valor_empenho


Empenhos sem liquidação simbolizariam, a principio, serviços não prestados ou bens não entregues até o momento. Porém, quando observamos a falta de liquidação em empenhos que foram pagos, percebemos que pode haver inconsistências no fluxo da despesa pública. Esse raciocínio guiará a consulta na seção de Detecção de Anomalias.

In [7]:
query_empenhos_sem_liquidacao = load_query('queries_base/empenhos_sem_liquidacao.sql')

df_empenhos_sem_liquidacao = run_query(query_empenhos_sem_liquidacao)
print(f"Empenhos sem liquidação: {len(df_empenhos_sem_liquidacao)}")
df_empenhos_sem_liquidacao.head()

Empenhos sem liquidação: 40


,id_empenho,valor_empenho
0,EMP-156,75599.37
1,EMP-195,74360.40
2,EMP-211,64634.91
3,EMP-130,64361.52
4,EMP-175,56580.02


É interessante avaliar o percentual de empenhos que foram, de fato, liquidados e pagos.


In [8]:
query_empenho_liquidacao = load_query('queries_base/percentual_empenho_liquidado.sql')

df_empenho_liquidacao = run_query(query_empenho_liquidacao)
num_empenhos_nao_liquidados = df_empenho_liquidacao['valor_liquidado'].isnull().sum()
num_empenhos_total = df_empenho_liquidacao.shape[0]
percentual_nao_liquidados = num_empenhos_nao_liquidados / num_empenhos_total * 100

print(f"Percentual de empenhos não liquidados: {percentual_nao_liquidados:.2f}%")
print(f"Percentual de empenhos liquidados: {100 - percentual_nao_liquidados:.2f}%")


Percentual de empenhos não liquidados: 8.05%
Percentual de empenhos liquidados: 91.95%


In [9]:
query_empenho_pagamento = load_query('queries_base/percentual_empenho_pago.sql')

df_empenho_pagamento = run_query(query_empenho_pagamento)
num_empenhos_nao_pagos = df_empenho_pagamento['valor_pagamento'].isnull().sum()
num_empenhos_total = df_empenho_pagamento.shape[0]
percentual_nao_pagos = num_empenhos_nao_pagos / num_empenhos_total * 100
print(f"Percentual de empenhos não pagos: {percentual_nao_pagos:.2f}%")
print(f"Percentual de empenhos pagos: {100 - percentual_nao_pagos:.2f}%")

Percentual de empenhos não pagos: 0.00%
Percentual de empenhos pagos: 100.00%


O resultado das duas ultimas células traz uma inconsistencia no fluxo da despesa publica, uma vez que 100% dos empenhos foram pagos, porém apenas 91.95% foram liquidados.

### 3.2 Detecção de inconsistências

Abaixo, apresento uma análise das inconsistências encontradas nos dados.

#### 3.2.1. Pagamentos > Valor Empenhado

Situação crítica onde o valor pago supera o que foi reservado (empenhado).

In [10]:
query_pagamentos_maior_empenho = load_query('queries_base/pagamentos_maior_empenho.sql')

df_pagamentos_maior_empenho = run_query(query_pagamentos_maior_empenho)
print(f"Empenhos com pagamento superior ao valor: {len(df_pagamentos_maior_empenho)}")
df_pagamentos_maior_empenho.head()

Empenhos com pagamento superior ao valor: 225


,valor_empenhado,total_pago,objeto_contrato,nome_fornecedor
0,26274.91,28910.00,Merenda Escolar - Lote 1,Saude Vida Distribuidora
1,2084.35,22827.97,Servico de Vigilancia 24h,TI Inovacao
2,6306.38,29566.53,Pavimentacao Rua X,Papelaria Central
3,10983.92,37385.87,Manutencao de Ar Condicionado,TechServer Solucoes
4,12906.43,40955.94,Reforma de Creche,Merenda Boa Alimentos


#### 3.2.2. Pagamento anterior ao Empenho
A legislação exige que o empenho preceda o pagamento.

In [11]:
query_datas_invalidas = load_query('queries_base/empenhos_datas_invalidas.sql')

df_datas_invalidas = run_query(query_datas_invalidas)
print(f"Pagamentos realizados antes do empenho: {len(df_datas_invalidas)}")

# média da diferença de dias entre pagamento e empenho
diferenca_dias = (pd.to_datetime(df_datas_invalidas['data_empenho']) - pd.to_datetime(df_datas_invalidas['data_pagamento'])).dt.days
print(f"Média da diferença de dias entre pagamento e empenho: {diferenca_dias.mean():.2f}")

df_datas_invalidas.head()

Pagamentos realizados antes do empenho: 41
Média da diferença de dias entre pagamento e empenho: 5.00


,id_empenho,data_empenho,data_pagamento,valor_pago,objeto_contrato
0,EMP-10,2024-01-13,2024-01-08,12856.77,Consultoria Financeira
1,EMP-112,2024-01-16,2024-01-11,21058.66,Locacao de Veiculos
2,EMP-127,2024-01-14,2024-01-09,33792.94,Manutencao de Ar Condicionado
3,EMP-14,2024-01-10,2024-01-05,37386.57,Servico de Vigilancia 24h
4,EMP-143,2024-01-11,2024-01-06,23335.29,Reforma de Creche


Uma consulta melhor ainda seria categorizar esses empenhos pelo objeto de contrato e aglutinar o total pago nessas despesas

In [12]:
query_empenhos_datas_invalidas_por_categoria = load_query('queries_base/empenhos_datas_invalidas_por_categoria.sql')

df_empenhos_datas_invalidas_por_categoria = run_query(query_empenhos_datas_invalidas_por_categoria)
print(f"Pagamentos realizados antes do empenho: {len(df_empenhos_datas_invalidas_por_categoria)}")

df_empenhos_datas_invalidas_por_categoria.head()

Pagamentos realizados antes do empenho: 9


,valor_pago,objeto_contrato,diferenca_datas
0,70242.89,Fornecimento de Papel A4,5
1,42579.03,Medicamentos Basicos,5
2,205755.74,Locacao de Veiculos,5
3,87110.65,Consultoria Financeira,5
4,90222.61,Reforma de Creche,5


#### 3.2.3 Empenhos pagos sem liquidação

Categorias com maior número de empenhos pagos sem liquidação:

In [13]:
query_empenhos_pagos_sem_liquidacao_por_objeto = load_query('queries_base/empenhos_pagos_sem_liquidacao_por_objeto.sql')

df_empenhos_pagos_sem_liquidacao_por_objeto = run_query(query_empenhos_pagos_sem_liquidacao_por_objeto)
print(f"Empenhos pagos sem liquidação: {df_empenhos_pagos_sem_liquidacao_por_objeto['numero_ocorrencias'].sum()}")
df_empenhos_pagos_sem_liquidacao_por_objeto.head()

Empenhos pagos sem liquidação: 40


,valor_total_pago,objeto_contrato,numero_ocorrencias
0,165638.96,Servico de Vigilancia 24h,7
1,165421.41,Manutencao de Ar Condicionado,7
2,138915.44,Locacao de Veiculos,4
3,116960.18,Consultoria Financeira,4
4,85286.93,Licencas de Software,4


## 4. Visualização Detalhada dos Resultados
Para visualização dos dados foi criado um dashboard no Google Looker Studio.

Link: [Google Looker Studio](https://lookerstudio.google.com/reporting/3a945586-fd2a-4d3c-9d47-0aadddd61cb0)